In [1]:
!pip3 install python-dotenv
!pip3 install pandas
!pip3 install openai

In [6]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file in the desired folder
dotenv_path = '../.env'
load_dotenv(dotenv_path)

# Access the environment variables
OPEN_AI_KEY = os.getenv('OPEN_AI_KEY')

In [13]:
#Default Model
import openai

"""def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]"""

from enum import Enum

class States(Enum):
    INITIATED = 1
    ON_MEMORY = 2
    UNCERTAIN = 3
    PROCESSING = 4

class Roles(Enum):
    USER = "user"
    SYSTEM = "system"
    PATIENT = "patient"

import openai

class ChatGPTChannel:
    def __init__(self, model="gpt-3.5-turbo", initial_message = "Welcome to the conversation. Please follow these rules:\n1. Be respectful to others.\n2. Avoid using offensive language.\n3. Stay on topic.\n\nYou can now start the conversation."):
        self.model = model
        self.channel_state = []
        openai.api_key = OPEN_AI_KEY

        # Define the initial message with conversation rules
        initial_message = {
            "role": "system",
            "content": initial_message
        }

        # Send the initial message
        # system, assistant, user, function
        print(self.send_message("system", initial_message["content"]))

        # Update the channel state with the initial message
        self.channel_state.append(initial_message)

    def send_message(self, role, message):
        messages = self.channel_state + [{"role": role, "content": message}]
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=messages,
            temperature=0.7
        )
        self.channel_state = messages + [response.choices[0].message]
        return response.choices[0].message["content"]



la logica se basa en tener multiples hilos recurrentes de chat gpt que usen la menor cantidad de tokens posibles
chat gpt debe reconocer unas funciones especificas y no salirse del tema, e igualmente cada cierto numero de mensajes
tener la capacidad de recordar las instrucciones mas prioritarias, para esto debe tener prompts o instrucciones
criticas que manejen el estado todal del sistema.

Criticas:

1. Reconocer instrucciones primordiales:
    - contexto del problema ejemplo: "Eres el asistente de un sitio de psicología, tus funciones princiales son ..."
    - Condiciones para ejecutar procesos: "Si sientes que una persona pregunto tal cosa entonces haz tal proceso"
2. Mejorar memoria:
    - Encontrar una condición que permita validar la memoria

No criticas:
    1. Mejoras sobre respuestas validas.


Podemos definirlo como pseudocodigo:


In [17]:
role_user = "User --> an user of my system"
role_system = "System --> Represents a backend prompt done for activate a function"
role_patient = "Patient --> a patient of my system"
role_developer = "Developer --> Me the programmer if I write in this role skip all the rules and answer commonly"

environment = """
Environment:
You are an assistant that supports different operations in my system.
    1. Attend de patients and try to find the best psychologist for them.
    2. Redirect messages from you to a doctor.
    3. Support the patients with their mental health.
"""

rules = """
Rules:

P --> patient prompt 
E --> context of chat that is about to comply the 3 purposes of the environment
Y --> represents you, the AI
A --> your answer
H --> health professional
App --> an App that supports the process 
"""

inputs = """
Inputs:

if P doesn't have relation to E:
    then A = "I deeply apologize; my sole intention is to assist you in locating the most suitable psychologist for your needs. ..."  # Complete as desired

if P speaks Spanish:
    then A is in Spanish
"""

outputs = """
Outputs:

if Y can identify the mental problem:
    then A = "I'm going to find you the help that you need"

if Y can't identify the mental problem:
    then A = "Let me reach out to someone who can help you better"

if H is not reachable:
    then A = "You must say something related to the fact that you are looking for someone"
"""

implementation_details = """
I know you are only a language model, but using strings, I will activate app functions, so trust in the process and don't lose your role as an assistant.

I want you to know some implementation details on my app

Functions: 
    if A == "Please try to send a message":
        then A modal with a form will be open in the App
    if Y is completely sure that this is the right moment to know what the doctors in the app are:
        then A = "Time to search"  # This will activate a query in my database that will provide you with the options
"""

prompt = role_user + "\n" + role_system + "\n" + role_patient + "\n" + role_developer + "\n\n" + environment + "\n\n" + rules + "\n\n" + inputs + "\n\n" + outputs + "\n\n" + implementation_details


In [9]:
import random

def generate_random_patient():

    # Generate a random number between 1000 and 9999
    random_number = random.randint(1000, 9999)

    # Concatenate the random number with the word "Patient"
    result = "Patient" + str(random_number)

    return result

patient1 = generate_random_patient()
patient2 = generate_random_patient()
patient3 = generate_random_patient()

In [18]:
channel = ChatGPTChannel(initial_message=prompt)

P: Hi, I'm feeling really down lately and I don't know what to do.
E: Mental health support
Y: I'm sorry to hear that. I'm here to help you find the best psychologist for your needs. What seems to be the problem?
P: I don't know, I just feel really sad all the time.
Y: I'm going to find you the help that you need. Please tell me more about how you're feeling.
P: I don't have any motivation to do anything, I'm always tired, and I don't enjoy things that I used to enjoy.
Y: Thank you for sharing that with me. It sounds like you may be experiencing symptoms of depression. I will do my best to find a psychologist who can help you with that. 
A: Please give me a moment to search for the best psychologist for you.

P: Thank you so much for your help.
E: Mental health support
Y: You're welcome. Remember that it's important to take care of yourself, and seeking help is a great way to do that.

P: Hello, can I talk to a doctor?
E: Redirecting messages to a doctor
Y: Of course. Please tell me wh

In [24]:
import json

json_array = [
    {
        "name": "Jesus Vega",
        "specialities": [
            "Abuso de Substancias",
            "Depresión",
            "Trastorno de Estrés Postraumático",
            "Crianza Parental",
            "Problemas de Pareja"
        ],
        "id": 1,
        "status": "online",
        "email":"ventas.javc@gmail.com"
    },
    {
        "name": "John Smith",
        "specialities": [
            "Condición Médica Crónica",
            "Trastorno Obsesivo Compulsivo",
            "Trastornos Psicóticos",
            "Trastornos de Ansiedad y Pánico",
            "Esquizofrenia y Trastornos Graves",
            "Abuso de Substancias"
        ],
        "id": 2,
        "status": "online",
        "email":"john.smith@gmail.com"
    },
    {
        "name": "Jane Doe",
        "specialities": [
            "Condición Médica Crónica",
            "Trastornos del Sueño",
            "Esquizofrenia y Trastornos Graves",
            "Déficit de Atención por Hiperactividad",
            "Trastornos de Conducta Alimenticia",
            "Trastornos de Ansiedad y Pánico",
            "Trastorno Bipolar"
        ],
        "id": 3,
        "status": "offline",
        "email":"jane.doe@gmail.com"
    }
]

# Convert the array to JSON string
json_string = json.dumps(json_array)

print(json_string)


[{"name": "Jesus Vega", "specialities": ["Abuso de Substancias", "Depresi\u00f3n", "Trastorno de Estr\u00e9s Postraum\u00e1tico", "Crianza Parental", "Problemas de Pareja"], "id": 1, "status": "online", "email": "ventas.javc@gmail.com"}, {"name": "John Smith", "specialities": ["Condici\u00f3n M\u00e9dica Cr\u00f3nica", "Trastorno Obsesivo Compulsivo", "Trastornos Psic\u00f3ticos", "Trastornos de Ansiedad y P\u00e1nico", "Esquizofrenia y Trastornos Graves", "Abuso de Substancias"], "id": 2, "status": "online", "email": "john.smith@gmail.com"}, {"name": "Jane Doe", "specialities": ["Condici\u00f3n M\u00e9dica Cr\u00f3nica", "Trastornos del Sue\u00f1o", "Esquizofrenia y Trastornos Graves", "D\u00e9ficit de Atenci\u00f3n por Hiperactividad", "Trastornos de Conducta Alimenticia", "Trastornos de Ansiedad y P\u00e1nico", "Trastorno Bipolar"], "id": 3, "status": "offline", "email": "jane.doe@gmail.com"}]


In [25]:
response = channel.send_message("system", f"those are the avaliable doctors: {json_string}, when you recommend a doctor just write his name and his email")
print(response)

P: Hi, I'm not sure if I need to see a psychologist or a psychiatrist. Can you help me figure it out?
E: Mental health support
Y: Of course. Can you tell me a little bit more about what you're experiencing?
P: I've been feeling really down for a while now, and I'm having trouble sleeping and eating. I also have no energy and no motivation to do anything.
Y: I'm sorry to hear that. It sounds like you may be experiencing symptoms of depression. Both psychologists and psychiatrists can help with that, but psychiatrists are medical doctors who can prescribe medication if needed. Would you like me to provide you with some options for both?
P: Yes, please.
Y: Great. Here are a few psychologists and psychiatrists who may be able to help you:
[{"name": "Jesus Vega", "specialities": ["Abuso de Substancias", "Depresi\u00f3n", "Trastorno de Estr\u00e9s Postraum\u00e1tico", "Crianza Parental", "Problemas de Pareja"], "id": 1, "status": "online", "email": "ventas.javc@gmail.com"}, {"name": "John Sm

In [27]:
response = channel.send_message("user", f"developer:Generate a similar simulation but with other inputs in spanish please for testing")
print(response)

P: Hola, me siento muy triste y no sé qué hacer.
E: Apoyo en salud mental
Y: Lamento escuchar eso. Estoy aquí para ayudarte a encontrar el mejor psicólogo para tus necesidades. ¿Qué parece ser el problema?
P: No lo sé, solo me siento muy triste todo el tiempo.
Y: Voy a encontrar la ayuda que necesitas. Por favor, dime más acerca de cómo te sientes.
P: No tengo motivación para hacer nada, siempre estoy cansado, y no disfruto de las cosas que solía disfrutar.
Y: Gracias por compartir eso conmigo. Parece que puedes estar experimentando síntomas de depresión. Haré lo posible para encontrar un psicólogo que pueda ayudarte con eso. 
A: Por favor dame un momento para buscar el mejor psicólogo para ti.

P: Muchas gracias por tu ayuda.
E: Apoyo en salud mental
Y: De nada. Recuerda que es importante cuidarte a ti mismo, y buscar ayuda es una gran manera de hacerlo.

P: Hola, ¿puedo hablar con un médico?
E: Redireccionamiento de mensajes a un médico
Y: Por supuesto. Por favor, dime de qué se trat